In [1]:
from glob import glob
import xarray as xr
import cf_xarray # use cf-xarray so that we can use CF attributes
import pandas as pd
import matplotlib.pyplot as plt
import regionmask
from xclim.indices.stats import frequency_analysis
import cartopy.crs as ccrs
import nc_time_axis
import numpy as np
import warnings
# To access collection
import dask
import intake
from dask_jobqueue import PBSCluster
from dask.distributed import Client, LocalCluster, futures_of
from dask.diagnostics import ProgressBar
from tqdm import tqdm 
import regionmask

## CESM2 CMIP6

### Spin up Dask cluster

In [2]:
# Create our NCAR Cluster - which uses PBSCluster under the hood
num_jobs = 10
cluster = PBSCluster(
    job_name = 'valencig-dask-hpc',
    cores = 1,
    memory = '10GiB',
    processes = 1,
    local_directory = '/glade/u/home/valencig/spilled/',
    log_directory = '/glade/u/home/valencig/worker-logs/',
    resource_spec = 'select=1:ncpus=1:mem=15GB',
    queue = 'casper',
    walltime = '02:00:00', # Change wall time if needed
    interface = 'ext'
)


# Spin up workers
cluster.scale(jobs=num_jobs)

# Assign the cluster to our Client
client = Client(cluster)

# Block progress until workers have spawned
client.wait_for_workers(num_jobs)
client

/glade/u/home/valencig/.conda/envs/wind/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46697 instead
  warnings.warn(


<Client: 'tcp://128.117.208.103:36159' processes=6 threads=6, memory=60.00 GiB>

### [Commands for managing dask workers](https://arc.ucar.edu/knowledge_base/68878389)


In [5]:
# See the workers in the job scheduler
!qstat -u $USER

# Kill all running or pending jobs
# !qdel `qselect -u $USER`

                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
9383328.casper* valencig htc      STDIN       61039   1   1   10gb 02:00 R 00:03
9383374.casper* valencig htc      valencig-*  62302   1   1   15gb 02:00 R 00:00
9383375.casper* valencig htc      valencig-*  62339   1   1   15gb 02:00 R 00:00
9383376.casper* valencig htc      valencig-*  46815   1   1   15gb 02:00 R 00:00
9383377.casper* valencig htc      valencig-*   1798   1   1   15gb 02:00 R 00:00
9383378.casper* valencig htc      valencig-*   3000   1   1   15gb 02:00 R 00:00
9383379.casper* valencig htc      valencig-*   3133   1   1   15gb 02:00 R 00:00
9383380.casper* valencig htc      valencig-*   3167   1   1   15gb 02:00 R 00:00
9383381.casper* valencig htc      valencig-*   3192   1   1   15gb 02:00 R 00:00
9383382.casper* valencig htc  

### Read in the catalog

In [6]:
# 'cesm.json' is copy of '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
# Comment out "options": null in aggregation_controls.aggregations.0 in order to get intake-esm to work
# cat = intake.open_esm_datastore('cesm2.json')
cat = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cmip6.json')
cat

,unique
activity_id,17
institution_id,35
source_id,81
experiment_id,130
member_id,421
table_id,36
variable_id,1075
grid_label,12
dcpp_init_year,59
version,597


### Querying for desired variable

https://www.cesm.ucar.edu/community-projects/lens/data-sets

CMIP6 variable list --> https://na-cordex.org/variable-list.html

CMIP6 variable list --> https://clipc-services.ceda.ac.uk/dreq/mipVars.html

Also --> https://wcrp-cmip.github.io/CMIP6_CVs/docs/CMIP6_experiment_id.html

In [7]:
cat.search(variable_id=['wind*', 'Wind*']).df.variable_id.unique()

array(['sfcWind', 'sfcWindmax', 'sndmasswindrif'], dtype=object)

### Query and subset data catalog

Overview found [here](https://www2.cesm.ucar.edu/projects/CMIP6/):

ScenarioMIP: "Will provide multi-model climate projections based on alternative scenarios of future emissions and land use changes produced with integrated assessment models. The design consists of eight alternative 21st century scenarios plus one large initial condition ensemble and a set of long-term extensions. Climate model projections will facilitate integrated studies of climate change as well as address targeted scientific questions."

Citation: O'Neill, B. C., Tebaldi, C., van Vuuren, D.P., Eyring, V., Friedlingstein, P., Hurtt, G., Knutti, R., Kriegler, E., Lamarque, J.-F., Lowe, J., Meehl, G.A., Moss, R., Riahi, K., and Sanderson, B. M. 2016. The Scenario Model Intercomparison Project (ScenarioMIP) for CMIP6. Geosci. Model Dev., 9, 3461-3482.

In [8]:
cesm2 = cat.search(
    variable_id='sfcWind', # near surface wind
    source_id='CESM2',
    experiment_id='ssp*',
    # experiment_id='historical', # all historical forcings
    table_id='day', # day is highest resolution
    activity_id='ScenarioMIP'
)
cesm2LE2.serialize(name="CESM2_CMIP6_ssp_sfcWind", catalog_type="file")

In [9]:
cesm2.keys_info()

,activity_id,institution_id,source_id,experiment_id,table_id,grid_label
key,,,,,,
ScenarioMIP.NCAR.CESM2.ssp126.day.gn,ScenarioMIP,NCAR,CESM2,ssp126,day,gn
ScenarioMIP.NCAR.CESM2.ssp245.day.gn,ScenarioMIP,NCAR,CESM2,ssp245,day,gn
ScenarioMIP.NCAR.CESM2.ssp370.day.gn,ScenarioMIP,NCAR,CESM2,ssp370,day,gn
ScenarioMIP.NCAR.CESM2.ssp585.day.gn,ScenarioMIP,NCAR,CESM2,ssp585,day,gn


### Read in using ```.to_dataset_dict()```

In [10]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = cesm2.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Let’s take a look at the keys - these are defined by the `groupby` attributes in the catalog. The groupby attributes in this case are:

https://www2.cesm.ucar.edu/experiments/cesm1.2/GLE/GLENS_output_fields/

`component.experiment.stream.forcing_variant.control_branch_year.variable`

- Component - which component this output is from (ex. atm represents the atmosphere)
- Experiment - which experiment this is from, in this case, this is `ssp370` which is one of the CMIP6 future experiments
- Stream - which stream this output is from, in this case, this is `cam.h1`, which represents daily output
- Control Branch Year - which year the ensemble branched off from, these are described within the [CESM2-LE documentation page](https://www.cesm.ucar.edu/community-projects/lens2)
- Variable - which variable you are working with


component = atm (atmosphere), lnd (land), ocn (ocean), ice

frequency = monthly, daily, or hourly6

experiment = historical (1850 to 2015) or ssp370 (2015 to 2100)

forcing_variant = the biomass forcing variant, cmip6 (the default in the cmip6 runs) or smbb (smoothed biomass burning)

variable = one of the variable names listed in the tables below

### Process Data

Time period for historical data is `1978` till `2014`.

Using the  annual maximum method with Gumbel distribution from [Palutikof et al., 1999](https://rmets.onlinelibrary.wiley.com/doi/pdfdirect/10.1017/s1350482799001103). Sourced from [this paper](https://meetingorganizer.copernicus.org/ECSS2019/ECSS2019-218-3.pdf) from the European Conference on Storms (2019).

In [11]:
def subset_ds(ds, task):
    states = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50
    # Hawaii and Alaska are not included in the mask
    good_keys = [
        k for k in states.regions.keys() 
        if k not in states.map_keys(['Hawaii', 'Alaska'])
    ]
    mask = states.mask(ds.lon, ds.lat).isin(good_keys)
    da = ds.where(mask, drop=True).sfcWind.sel(time=slice('2015', '2100'))  # Last time (2100) is wonky
    if task == 'mean':
        result = da.resample(time='1Y').mean(dim='time')
    elif task == 'anomaly':
        # Anomaly is x-x_mean
        # Then get average anomaly on a yearly basis
        result = (da - da.mean('time')).resample(time='1Y').mean('time')
    return result

Tasks are `mean` or `anomaly` or `50yr`
- Mean: yearly mean
- Anomaly: yearly anomaly
- 50yrmax: Gets 50 year max estimates for 2021–2040 (the “near-term”), 2041–2060 (the “mid-term”), and 2081–2100 (the “long-term”)

In [13]:
tasks = ('mean', 'anomaly')
for task in tasks:
    print(f'Running task: {task}')
    for key in tqdm(list(dsets.keys()), desc='Processing Data...'):
        ds = dsets[key].chunk({'time': 365})  # Get into approx 100 mb chunks
        da = subset_ds(ds, task=task).persist()
        da.compute().to_netcdf('/glade/u/home/valencig/wind-trend-analysis/data/'+key+'.'+task+'.nc')

Running task: 50yrmax


Processing Data...: 100%|██████████| 4/4 [03:13<00:00, 48.37s/it]


## CESM2 Large Ensemble 2

### Extract projected surface wind speeds

In [5]:
# 'cesm.json' is copy of '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
# Comment out "options": null in aggregation_controls.aggregations.0 in order to get intake-esm to work
cat = intake.open_esm_datastore('cesm2.json')
cesm2LE2wind = cat.search(
    long_name='Horizontal total wind speed average at the surface',  # maximum daily near surface wind speed
    experiment='ssp370',  # future projection, alternative is 'historical
    frequency='day_1',
    forcing_variant='smbb'
)
cesm2LE2wind.serialize(name="CESM2_LE2_WSPDSRFAV", catalog_type="file")
cesm2LE2wind.keys_info()

/glade/u/home/valencig/.conda/envs/wind/lib/python3.11/site-packages/intake_esm/cat.py:251: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


Successfully wrote ESM catalog json file to: file:///glade/u/home/valencig/wind-trend-analysis/CESM2_LE2_WSPDSRAV.json


,component,experiment,stream,forcing_variant,variable
key,,,,,
atm.ssp370.cam.h1.smbb.WSPDSRFAV,atm,ssp370,cam.h1,smbb,WSPDSRFAV


In [7]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = cesm2LE2wind.to_dataset_dict(aggregate=False)
    all_files = list(dsets.values())
    # Extract variable of interest
    max_winds = [f.WSPDSRFAV for f in all_files]
    ds = xr.combine_by_coords(max_winds)

In [17]:
def extractWindAnomaly(ds):
    states = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50
    # Hawaii and Alaska are not included in the mask
    good_keys = [
        k for k in states.regions.keys()
        if k not in states.map_keys(['Hawaii', 'Alaska'])
    ]
    mask = states.mask(ds.lon, ds.lat).isin(good_keys)
    da = ds.where(mask, drop=True).WSPDSRFAV.sel(time=slice('2015', '2100'))
    result = (da - da.mean('time')).resample(time='1Y').mean('time')
    return result

In [18]:
ds = ds.chunk({'time': 365})  # Get into approx 100 mb chunks
da = extractWindAnomaly(ds).persist()
da.compute().to_netcdf('/glade/u/home/valencig/wind-trend-analysis/data/atm.ssp370.cam.h1.smbb.WSPDSRFAV.anom.nc')

### Get data for 50 Year max

Variable ```sfcWindmax``` (Daily maximum near surface winds) not available for CESM2 for CMIP6 runs. Using the CESM2 Large Ensemble 2 instead (ssp370).

In [3]:
# 'cesm.json' is copy of '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
# Comment out "options": null in aggregation_controls.aggregations.0 in order to get intake-esm to work
cat = intake.open_esm_datastore('cesm2.json')
cat.search(long_name=['wind*', 'Wind*']).df.long_name.unique()

/glade/u/home/valencig/.conda/envs/wind/lib/python3.11/site-packages/intake_esm/cat.py:251: DtypeWarning: Columns (5,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


array(['Zonal wind', 'Zonal wind at   10 mbar pressure surface',
       'Zonal wind at   50 mbar pressure surface', '10m wind speed',
       'Zonal wind at  100 mbar pressure surface',
       'Zonal wind at 1000 mbar pressure surface',
       'Zonal wind at 200 mbar pressure surface',
       'Zonal wind at  200 mbar pressure surface',
       'Zonal wind at 500 mbar pressure surface',
       'Zonal wind at  500 mbar pressure surface',
       'Zonal wind at 700 mbar pressure surface',
       'Zonal wind at  700 mbar pressure surface',
       'Zonal wind at 850 mbar pressure surface',
       'Zonal wind at  850 mbar pressure surface',
       'Lowest model level zonal wind',
       'Zonal-Mean zonal wind - defined on ilev', 'Meridional wind',
       'Meridional wind at   10 mbar pressure surface',
       'Meridional wind at   50 mbar pressure surface',
       'Meridional wind at  100 mbar pressure surface',
       'Meridional wind at 1000 mbar pressure surface',
       'Meridional wind at 

Forcing variant is either cmip6 biomass burning (```cmip6```) or smoothed biomass burning (```smbb```). Read more [here](https://www.cesm.ucar.edu/community-projects/lens2)

In [4]:
cesm2LE2gust = cat.search(
    long_name='Horizontal total wind speed maximum at the surface',  # maximum daily near surface wind speed
    experiment='ssp370',  # future projection, alternative is 'historical
    frequency='day_1',
    forcing_variant='smbb'
)
cesm2LE2gust.serialize(name="CESM2_LE2_WSPDSRFMX", catalog_type="file")

Successfully wrote ESM catalog json file to: file:///glade/u/home/valencig/wind-trend-analysis/CESM2_LE2_WSPDSRFMX.json


In [14]:
cesm2LE2gust.keys_info()

,component,experiment,stream,forcing_variant,variable
key,,,,,
atm.ssp370.cam.h1.smbb.WSPDSRFMX,atm,ssp370,cam.h1,smbb,WSPDSRFMX


In [6]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = cesm2LE2gust.to_dataset_dict(aggregate=False)
    all_files = list(dsets.values())
    # Extract variable of interest
    max_winds = [f.WSPDSRFMX for f in all_files]
    ds = xr.combine_by_coords(max_winds)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.stream.case.member_id.variable.start_time.end_time.time_range.long_name.units.vertical_levels.frequency.path.experiment.forcing_variant.cesm_member_id.control_branch_year.cmip_experiment_id'


In [7]:
def get50yrmax(ds):
    states = regionmask.defined_regions.natural_earth_v5_0_0.us_states_50
    # Hawaii and Alaska are not included in the mask
    good_keys = [
        k for k in states.regions.keys() 
        if k not in states.map_keys(['Hawaii', 'Alaska'])
    ]
    mask = states.mask(ds.lon, ds.lat).isin(good_keys)
    da = ds.where(mask, drop=True).WSPDSRFMX.sel(time=slice('2015', '2100'))  # Last time (2100) is wonky
    near = frequency_analysis(
        da.sel(time=slice('2021', '2040')),
        t=50,
        dist="genextreme",
        mode="max",
        freq="YS"
    ).sel(return_period=50).assign_coords({'forecast': 'near'}).expand_dims('forecast')
    mid = frequency_analysis(
        da.sel(time=slice('2041', '2060')),
        t=50,
        dist="genextreme",
        mode="max",
        freq="YS"
    ).sel(return_period=50).assign_coords({'forecast': 'mid'}).expand_dims('forecast')
    far = frequency_analysis(
        da.sel(time=slice('2081', '2100')),
        t=50,
        dist="genextreme",
        mode="max",
        freq="YS"
    ).sel(return_period=50).assign_coords({'forecast': 'far'}).expand_dims('forecast')
    result = xr.combine_by_coords([near, mid, far])
    return result

In [ ]:
ds = ds.chunk({'time': 365})  # Get into approx 100 mb chunks
da = get50yrmax(ds).persist()

In [15]:
da.compute().to_netcdf('/glade/u/home/valencig/wind-trend-analysis/data/atm.ssp370.cam.h1.smbb.WSPDSRFMX.50yrWSPDSRFMX.nc')

## Restart dask cluster

In [ ]:
#client.restart()

## Close dask cluster

In [ ]:
client.shutdown()